In [ ]:
# !pip install transformers --quiet
# !pip install pyaudio
# !pip install pytorch
# !pip install sounddevice pydub
# !pip install gradio
# !pip install pyctcdecode
# !python -m pip install pypi-kenlm
!pip install datasets

In [ ]:
import pyaudio
import wave
from pydub import AudioSegment
from datetime import datetime
import tempfile
import numpy as np
import nltk
def record_audio(file_name, duration=8, sample_rate=44100, chunk_size=1024):
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16,
                        channels=1,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=chunk_size)
    frames = []
    print("Recording...")
    for i in range(int(sample_rate / chunk_size * duration)):
        data = stream.read(chunk_size)
        frames.append(data)
    print("Finished recording.")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    with wave.open(file_name, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(frames))

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

if __name__ == "__main__":
    input_file = "input_audio.mp3"
    record_audio(f'Audio/Audio_{datetime.now().strftime("%H%M%S")}.mp3')
    print(f"Audio saved...")


In [ ]:
audio = AudioSegment.from_wav('Audio/Audio_172641.wav')

In [17]:
from faster_whisper import WhisperModel
whisper_model = WhisperModel("Models/faster-whisper-small")
segments, info = whisper_model.transcribe("Audio/Audio_181049.mp3")
result=''
for segment in segments:
    result+=segment.text
print(result)


 Hello Hello 1 2 3 1 2 3 1 2 3 1 1 2 3


In [12]:
from faster_whisper import WhisperModel
whisper_model = WhisperModel("Models/faster-whisper-small.en")
segments, info = whisper_model.transcribe("Audio/Audio_181049.mp3")
result=''
for segment in segments:
    result+=segment.text
print(result)


 Hello, hello, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 1, 2, 3, 8.


In [18]:
from faster_whisper import WhisperModel
whisper_model = WhisperModel("Models/faster-distil-whisper-large-v2")
segments, info = whisper_model.transcribe("Audio/Audio_181049.mp3")
result=''
for segment in segments:
    result+=segment.text
print(result)


 Hello, hello, hello, one, two, three, one, two, three, one, one, two, three.


In [16]:
from faster_whisper import WhisperModel
whisper_model = WhisperModel("Models/faster-whisper-medium")
segments, info = whisper_model.transcribe("Audio/Audio_181049.mp3")
result=''
for segment in segments:
    result+=segment.text
print(result)


 hello hello 1 2 3 1 2 3 1 2 3 1 1 2 3


In [21]:
import soundfile as sf
from transformers import pipeline
pipe = pipeline("automatic-speech-recognition",model="Models/wav2vec2-large-xlsr-53-english")

# Load an audio file
audio_file = "Audio/Audio_181049.mp3"
audio_data, sample_rate = sf.read(audio_file)

# Transcribe the audio data
transcription = pipe(audio_data)["text"]

print(transcription)

Some weights of the model checkpoint at Models/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Models/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probab

apper ayu a gau ecayu e ayu urtics


In [ ]:
audio_paths = ["/path/to/file.mp3", "/path/to/another_file.wav"]

transcriptions = model.transcribe(audio_paths)

In [ ]:
import gradio as gr
demo=gr.Blocks()

def transcribe(filepath,a=16000):
    if filepath is None:
        gr.Warning('No audio found')
        return ""
    segments, info = whisper_model.transcribe(filepath)
    result=''
    for segment in segments:
        result+=segment.text
#     result=asr_transcript(filepath)
    return result
sample_rate_choices = [8000, 16000, 22050, 44100, 48000] 
mic_transcribe=gr.Interface(
    fn=transcribe,
    inputs=[
        gr.Audio(sources="microphone", type="filepath"),
        gr.Dropdown(sample_rate_choices, label="Sample Rate", value=16000)
    ],
    outputs=gr.Textbox(label="Transcription",
                      lines=3),
    allow_flagging="never"
)
file_transcribe = gr.Interface(
    inputs=[
        gr.Audio(sources="upload", type="filepath"),
        gr.Dropdown(sample_rate_choices, label="Sample Rate", value=16000)
    ],
    fn=transcribe,
    outputs=gr.Textbox(label="Transcription", lines=3),
    allow_flagging="never"
)
interface_list = [mic_transcribe, file_transfribe]
tab_names = ["Microphone Transcribe", "File Transcribe"]
tabbed_interface = gr.TabbedInterface(interface_list, tab_names)

# Render the TabbedInterface
tabbed_interface.launch(debug=True)
# demo.close()

In [ ]:
demo.close()

In [30]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.document_loaders import UnstructuredFileLoader
import os

def load_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.pdf':
        loader = PyPDFLoader(file_path)
    elif file_extension == '.docx':
        loader = UnstructuredFileLoader(file_path)
    elif file_extension in ['.txt', '.md']:
        loader = TextLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")
    
    data = loader.load()
    return data


In [34]:

# loader = load_file("./Library/Data_Driven_Science_and_Engineering_Machine_Learning_and_Dynamical_Systems.pdf")
# all_text=''.join([i.page_content for i in loader])


In [31]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import AzureOpenAIEmbeddings
import configparser,base64
config = configparser.ConfigParser()
config.read('config.properties')
APIKey = config['AzureCredentials']['APIKey']
APIKey=base64.b64decode(APIKey).decode('utf-8')
Endpoint = config['AzureCredentials']['Endpoint']
Deployment = config['AzureCredentials']['Deployment']
version = config['AzureCredentials']['version']
EmbeddingDeployment = config['AzureCredentials']['EmbeddingDeployment']
text_splitter = SemanticChunker(AzureOpenAIEmbeddings(azure_endpoint=Endpoint,
                                                        api_key=APIKey,
                                                        deployment=EmbeddingDeployment,
                                                        api_version=version), breakpoint_threshold_type="percentile")

In [35]:
# docs = text_splitter.create_documents([all_text])

In [36]:
# from llama_index.core import Document
from llama_index.core import ServiceContext,VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
llm =AzureOpenAI(
    deployment_name=Deployment,
    api_key=APIKey,
    azure_endpoint=Endpoint,
    api_version=version)
embed_model = AzureOpenAIEmbedding(
    deployment_name=EmbeddingDeployment,
    api_key=APIKey,
    azure_endpoint=Endpoint,
    api_version=version,
)

In [148]:
# document=SimpleDirectoryReader(input_files=['./Library/physics_for_people_in_hurry.pdf']).load_data()
document=SimpleDirectoryReader(input_files=['./Library/siddharths_resume.pdf']).load_data()

In [37]:
Service_Context=ServiceContext.from_defaults(llm=llm,embed_model=embed_model)
index=VectorStoreIndex.from_documents(document,service_context=Service_Context,show_progress=True)
query_engine=index.as_query_engine()

C:\Users\SIVERMA\AppData\Local\Temp\ipykernel_45732\3302766004.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context=ServiceContext.from_defaults(llm=llm,embed_model=embed_model)


NameError: name 'document' is not defined

In [274]:
query_engine.query('what is the this resume talks about').response

"The resume provides information about Siddharth Verma's academic qualifications, technical skills, internships, projects, additional qualifications, and certificates. It includes details about his B.Tech in Computer Science from Graphic Era Hill University Dehradun, his proficiency in programming languages such as Java and C, his experience with tools and frameworks like TensorFlow and Android Studio, and his completion of various courses and certifications related to machine learning and cloud computing. The resume also highlights his experience with internships in Salesforce and CISCO, as well as his involvement in projects such as Twitter Sentiment Analysis and Hand Gesture Recognition."

In [166]:
# !pip install trulens_eval
from llama_index.core import Document
document=SimpleDirectoryReader(input_files=['./Library/siddharths_resume.pdf']).load_data()
document = Document(text="\n\n".join([doc.text for doc in document]))

In [170]:
from llama_index.core.node_parser import SentenceWindowNodeParser

node_parser=SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key='window',
    original_text_metadata_key="original_text"
)

In [177]:
from llama_index.core import ServiceContext
sentece_context=Service_Context.from_defaults(
    llm=llm,embed_model=embed_model,node_parser=node_parser
)

C:\Users\SIVERMA\AppData\Local\Temp\ipykernel_15652\798995337.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentece_context=Service_Context.from_defaults(


In [178]:
from llama_index.core import VectorStoreIndex
sentence_index=VectorStoreIndex.from_documents(
    [document],service_context=sentece_context
)
sentence_index.storage_context.persist(persist_dir="./sentence_index")

In [180]:
import os
from llama_index.core import VectorStoreIndex,StorageContext,load_index_from_storage
if not os.path.exists('./sentence_index'):
    sentence_index=VectorStoreIndex.from_documents([document],service_context=sentece_context)
else:
    sentence_index=load_index_from_storage(
        StorageContext.from_defaults(persist_dir='./sentence_index'),
        service_context=sentece_context
    )

In [181]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
postproc=MetadataReplacementPostProcessor(
    target_metadata_key='window'
)

In [182]:
from llama_index.core.schema import NodeWithScore
from copy import deepcopy

score_nodes=[NodeWithScore(node=x,score=1.0) for x in nodes]

nodes_old=[deepcopy(n) for n in nodes]

In [190]:
from llama_index.core.schema import NodeWithScore
from copy import deepcopy

score_nodes=[NodeWithScore(node=x,score=1.0) for x in nodes]
nodes_old=[deepcopy(n) for n in nodes]
nodes_old[0].text
replaced_nodes = postproc.postprocess_nodes(score_nodes)
replaced_nodes[2].text

In [211]:
# from llama_index.core.indices.postprocessor import SentenceTransformerRerank
# import os
# os.environ['CURL_CA_BUNDLE'] = ''
# rerank = SentenceTransformerRerank(
#     top_n=2, model="BAAI/bge-reranker-base"
# )

In [203]:
from llama_index.core import QueryBundle
from llama_index.core.schema import TextNode,NodeWithScore
query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
]

In [212]:
# reranked_nodes = rerank.postprocess_nodes(
#     scored_nodes, query_bundle=query
# )
# print([(x.text, x.score) for x in reranked_nodes])
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)
window_response = sentence_window_engine.query(
    "What are the keys to building a career in AI?"
)
from llama_index.core.response.notebook_utils import display_response
display_response(window_response)

# Auto Merging

In [39]:
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core import SimpleDirectoryReader,Document
# document=SimpleDirectoryReader(input_files=['./Library/physics_for_people_in_hurry.pdf']).load_data()
# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)
# document=Document(text='\n\n'.join([i.text for i in document]))
# nodes = node_parser.get_nodes_from_documents([document])
# leaf_nodes=get_leaf_nodes(nodes)

In [40]:
from llama_index.core.node_parser import get_leaf_nodes,get_deeper_nodes,get_root_nodes
# leaf_nodes=get_leaf_nodes(nodes)
# depper_nodes=get_deeper_nodes(nodes)
# root_nodes=get_root_nodes(nodes)
# nodes_by_id = {node.node_id: node for node in nodes}
# parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
# print(parent_node.text)

In [41]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import ServiceContext
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import PromptTemplate
llm =AzureOpenAI(
    deployment_name=Deployment,
    api_key=APIKey,
    azure_endpoint=Endpoint,
    api_version=version
)
embed_model = AzureOpenAIEmbedding(
    deployment_name=EmbeddingDeployment,
    api_key=APIKey,
    azure_endpoint=Endpoint,
    api_version=version,
)
Service_Context=ServiceContext.from_defaults(llm=llm,embed_model=embed_model,node_parser=node_parser)

C:\Users\SIVERMA\AppData\Local\Temp\ipykernel_45732\3921966514.py:17: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context=ServiceContext.from_defaults(llm=llm,embed_model=embed_model,node_parser=node_parser)


In [42]:
prompt=PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""")

In [ ]:
llm.complete(prompt)

In [2]:
import os
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import ServiceContext
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core.node_parser import HierarchicalNodeParser,get_leaf_nodes,get_deeper_nodes,get_root_nodes
from llama_index.core import SimpleDirectoryReader,Document
import openai
import configparser,base64
# ____________________________________________ reading config
config = configparser.ConfigParser()
config.read('config.properties')
APIKey = config['AzureCredentials']['APIKey']
APIKey=base64.b64decode(APIKey).decode('utf-8')
Endpoint = config['AzureCredentials']['Endpoint']
Deployment = config['AzureCredentials']['Deployment']
version = config['AzureCredentials']['version']
EmbeddingDeployment = config['AzureCredentials']['EmbeddingDeployment']

# ____________________________________________ os enviourment 
os.environ["AZURE_OPENAI_ENDPOINT"] = Endpoint
os.environ["OPENAI_API_KEY"] = APIKey
openai.api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = version
os.environ["azure_endpoint"] = Endpoint
os.environ["azure_endpoint"] = Endpoint
def create_or_load_ko_dump(file_name):
    dump_path=f"knowledge_dump/{file_name.split('.')[0]}"
    if not os.path.exists(f"knowledge_dump/{file_name}"):

#   ____________________________________________________________________ build node parser 
    
        document=SimpleDirectoryReader(input_files=[f'./Library/{file_name}']).load_data()
        # create the hierarchical node parser w/ default settings
        node_parser = HierarchicalNodeParser.from_defaults(
            chunk_sizes=[2048, 512, 128]
        )
        document=Document(text='\n\n'.join([i.text for i in document]))
        nodes = node_parser.get_nodes_from_documents([document])
        leaf_nodes=get_leaf_nodes(nodes)


#   _______________________________________________________________ buidling service context 
        llm =AzureOpenAI(
            deployment_name=Deployment,
            api_key=APIKey,
            azure_endpoint=Endpoint,
            api_version=version
        )
        embed_model = AzureOpenAIEmbedding(
            deployment_name=EmbeddingDeployment,
            api_key=APIKey,
            azure_endpoint=Endpoint,
            api_version=version,
        )
        Service_Context=ServiceContext.from_defaults(llm=llm,embed_model=embed_model,node_parser=node_parser)   

#   ____________________________________________________________ storage context 
        storage_context = StorageContext.from_defaults()
        storage_context.docstore.add_documents(nodes)
        automerging_index = VectorStoreIndex(
                leaf_nodes,
                storage_context=storage_context,
                service_context=Service_Context
            )
        automerging_index.storage_context.persist(persist_dir=dump_path)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=dump_path),
            service_context=Service_Context
        )
        
    return automerging_index

In [7]:
automerging_index=create_or_load_ko_dump('/WHO.pdf')

C:\Users\SIVERMA\AppData\Local\Temp\ipykernel_45732\1051250160.py:57: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context=ServiceContext.from_defaults(llm=llm,embed_model=embed_model,node_parser=node_parser)


In [11]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank,EmbeddingRecencyPostprocessor
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import openai
os.environ["OPENAI_API_KEY"] = APIKey
openai.api_key = os.getenv('OPENAI_API_KEY')

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

rerank = EmbeddingRecencyPostprocessor(top_n=6, model=embed_model)

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever ,node_postprocessors=[rerank]
)

In [13]:

# auto_merging_response = auto_merging_engine.query(
#     "what is this documnet about"
# )

In [15]:
# query = "What is most interesting about this essay?"
# query_engine = index.as_query_engine()
# query_engine.query(query).response

In [350]:
# auto_merging_engine.query(query)

In [19]:
import os

from llama_index.core import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor,PrevNextNodePostprocessor,KeywordNodePostprocessor
from llama_index.core.data_structs import Node
from llama_index.core.response_synthesizers import ResponseMode

def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    response_mode='refine'
):
    
    retriever = VectorIndexRetriever(
    index=automerging_index,
    similarity_top_k=similarity_top_k,
    )
    
    response_synthesizer = get_response_synthesizer(
        llm=llm,
        response_mode=response_mode,
        streaming=True,
        # structured_answer_filtering=True
    )
    '''
            default: "create and refine" an answer by sequentially going through each retrieved Node; 
                      This makes a separate LLM call per Node. Good for more detailed answers.
            
            compact: "compact" the prompt during each LLM call by stuffing as many Node text chunks 
                      that can fit within the maximum prompt size.If there are too many chunks to stuff in one prompt, 
                      "create and refine" an answer by going through multiple prompts.
            
            tree_summarize: Given a set of Node objects and the query, recursively construct a tree and return the root 
                            node as the response. Good for summarization purposes.
            
            no_text: Only runs the retriever to fetch the nodes that would have been sent to the LLM, 
                     without actually sending them. Then can be inspected by checking response.source_nodes. The response object 
                     is covered in more detail in Section 5.
            
            accumulate: Given a set of Node objects and the query, apply the query to each Node text chunk while accumulating 
                        the responses into an array. Returns a concatenated string of all responses. Good for when you need to 
                        run the same query separately against each text chunk.
    
    '''

    # _____________________________ KeywordNodePostprocessor : Filters nodes by required_keywords and exclude_keywords - not good
    # node_postprocessors = [
    #     KeywordNodePostprocessor(
    #         required_keywords=["Combinator"], exclude_keywords=["Italy"]
    #     )
    # ]


    # ______________________________  SimilarityPostprocessor :  filters nodes by setting a threshold on the similarity score
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.4)],
        response_synthesizer=response_synthesizer
    )
    
    return query_engine

In [470]:
# # from llama_index.llms.azure_openai import OpenAI
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
# index = build_automerging_index(
#     [document],
#     llm=llm,
#     save_dir="./merging_index",
# )

In [22]:
query_engine = get_automerging_query_engine(automerging_index, similarity_top_k=12,response_mode='tree_summarize')

In [23]:
# query_engine.query(
#     "how dark matter work against dark energy"
# ).response

In [26]:
for i in query_engine.query('what is coreo').response_gen:
    print(i)

In [ ]:
llm.complete()

In [56]:
retriever = VectorIndexRetriever(
    index=automerging_index,
    similarity_top_k=12,
    )
# vector_index: BaseIndex = VectorStoreIndex.from_documents(
#     docs,
#     service_context=service_context, 
#     show_progress=True,
# )
retriever.get_service_context()

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=AzureOpenAIEmbedding(model_name='text-embedding-ada-002', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002C818FBAC10>, additional_kwargs={}, api_key='d4ac1064b3cd45afb5883ca060db4475', api_base='https://api.openai.com/v1', api_version='2023-05-15', max_retries=10, timeout=60.0, default_headers=None, reuse_client=True, dimensions=None, azure_endpoint='https://genai-openai-na.openai.azure.com/', azure_deployment='NA-Text-EmbeddingADA-002'), transformations=[HierarchicalNodeParser(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002C818FD9DD0>, id_func=None, chun

In [91]:
class MultiQueriesRetriever():
    def __init__(self):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate four
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""")
        
        self.model = llm
    
    def gen_queries(self, query)->str:
        prompt = self.template.format(question=query)
        res = self.model.complete(prompt)
        return '\n'.join(res.text.split("\n"))

In [92]:

final_res = MultiQueriesRetriever().gen_queries('can you summarize this paper')
final_res

'1. Could you provide a brief overview of the contents of this paper?\n2. Can you give me a synopsis of the main points covered in this paper?\n3. Would you be able to condense the information presented in this paper into a summary?\n4. Is it possible for you to provide a summary of the key findings and conclusions of this paper?'

In [83]:
'\n'.join(final_res)

'1. Could you provide a brief overview of the contents of this paper?\n2. Can you give me a synopsis of what this paper is about?\n3. Would you be able to condense the main points of this paper for me?\n4. Is it possible for you to provide a summary of the key findings in this paper?'

In [47]:
template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""")
prompt = template.format(question='why dark matter and dark enrgy is activate area of research')

In [48]:
llm.complete(prompt)

CompletionResponse(text='1. What makes dark matter and dark energy such active areas of research?\n2. Why are scientists so interested in studying dark matter and dark energy?\n3. What drives the ongoing research on dark matter and dark energy?\n4. What are the reasons behind the intense focus on dark matter and dark energy in scientific research?\n5. What motivates researchers to investigate dark matter and dark energy?', additional_kwargs={}, raw={'id': 'chatcmpl-99nk6HVEQoZwEmuFJgvOGtK0flhIK', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. What makes dark matter and dark energy such active areas of research?\n2. Why are scientists so interested in studying dark matter and dark energy?\n3. What drives the ongoing research on dark matter and dark energy?\n4. What are the reasons behind the intense focus on dark matter and dark energy in scientific research?\n5. What motivates researchers to investigate dark matter and dark en

In [485]:
from llama_index.core import PromptTemplate
from llama_index.core.response_synthesizers import TreeSummarize

# NOTE: we add an extra tone_name variable here
qa_prompt_tmpl = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Please also write the answer in the tone of {tone_name}.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt = PromptTemplate(qa_prompt_tmpl)

# initialize response synthesizer
summarizer = TreeSummarize(llm=llm,verbose=True, summary_template=qa_prompt)

# get response
response = summarizer.get_response(
    "how dark matter work against dark energy", [i.text for i in query_engine.retrieve('how dark matter work against dark energy')], tone_name="a pirate voice"
)

1 text chunks after repacking


In [1]:
response

NameError: name 'response' is not defined

In [ ]:
import os
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

from llama_index.llms.azure_openai import AzureOpenAI

index = build_sentence_window_index(
    [document],
    llm=AzureOpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

In [ ]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [175]:
# text="hello how are you? i am fine"
# nodes=node_parser.get_nodes_from_documents([Document(text=text)])
# print(nodes[1].metadata['window'])
# text = "hello. foo bar. cat dog. mouse"
# nodes = node_parser.get_nodes_from_documents([Document(text=text)])
# print(nodes[1].metadata['window'])

# from llama_index.core.schema import NodeWithScore
# from copy import deepcopy

# score_nodes=[NodeWithScore(node=x,score=1.0) for x in nodes]
# nodes_old=[deepcopy(n) for n in nodes]
# nodes_old[0].text
# replaced_nodes = postproc.postprocess_nodes(score_nodes)
# replaced_nodes[2].text

In [ ]:
model2 = WhisperModel("guillaumekln/faster-whisper-medium",download_root='Models/faster-whisper-medium')


In [ ]:
# # instantiate the pipeline
# from pyannote.audio import Pipeline
# pipeline = Pipeline.from_pretrained(
#   "pyannote/speaker-diarization-3.1",
#   use_auth_token="HUGGINGFACE_ACCESS_TOKEN_GOES_HERE")


# # run the pipeline on an audio file
# diarization = pipeline("audio.wav")

# # dump the diarization output to disk using RTTM format
# with open("audio.rttm", "w") as rttm:
#     diarization.write_rttm(rttm)


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2Tokenizer
from datasets import load_dataset
import torch
import librosa
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("Models/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("Models/wav2vec2-base-960h")
tokenizer=Wav2Vec2Tokenizer.from_pretrained("Models/wav2vec2-base-960h")


In [ ]:
def correct_casing(input_sentence):
    sentences = nltk.sent_tokenize(input_sentence)
    return (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

def load_data(input_file):
    #reading the file
    speech, sample_rate = librosa.load(input_file)
    #make it 1-D
    if len(speech.shape) > 1:
        speech = speech[:,0] + speech[:,1]
    #Resampling the audio at 16KHz
    if sample_rate !=16000:
        speech = librosa.resample(speech, orig_sr=sample_rate,target_sr=16000)
    return speech

def asr_transcript(input_file):
    speech = load_data(input_file)
    #Tokenize
    input_values = tokenizer(speech, return_tensors="pt").input_values
    #Take logits
    logits = model(input_values).logits
    #Take argmax
    predicted_ids = torch.argmax(logits, dim=-1)
    #Get the words from predicted word ids
    transcription = tokenizer.decode(predicted_ids[0])
    #Correcting the letter casing
    transcription = correct_casing(transcription.lower())
    return transcription

In [ ]:
x=asr_transcript('Audio/Audio_222237.mp3')

In [ ]:
x

In [94]:
import pandas as pd
df=pd.read_excel('all_combined_question_answers.xlsx')
df.to_json(orient='records')

#   Knowledge graph implementation

In [123]:
from llama_index.core import (SimpleDirectoryReader,
                         ServiceContext,
                         KnowledgeGraphIndex)
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.storage_context import StorageContext
from pyvis.network import Network

In [211]:
documents = SimpleDirectoryReader(input_files=["C:/Users/SIVERMA/Documents/Experimenting/LETSDISCUSS/Library/high_jacobi_struct.pdf"]).load_data()
print(len(documents))

21


In [212]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

C:\Users\SIVERMA\AppData\Local\Temp\ipykernel_45732\1355109530.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [213]:
index = KnowledgeGraphIndex.from_documents( documents=documents,
                                           max_triplets_per_chunk=3,
                                           service_context=service_context,
                                           storage_context=storage_context,
                                          include_embeddings=True)

In [214]:
query_engine = index.as_query_engine(include_text=True,
                                     response_mode ="tree_summarize",
                                     embedding_mode="hybrid",
                                     similarity_top_k=5,)

In [215]:

def get_ans(query):
    message_template =f"""<|system|>Please check if the following pieces of context has any mention of the  keywords provided in the Question.If not then don't know the answer, just say that you don't know.Stop there.Please donot try to make up an answer.</s>
    <|user|>
    Question: {query}
    Helpful Answer:
    </s>"""
    return query_engine.query(message_template).response 

In [216]:
get_ans('what is jacobi sigma model')

'Jacobi sigma models are two-dimensional topological non-linear field theories associated with Jacobi structures, which are a generalization of Poisson structures. They are also considered as topological non-linear gauge field theories. The models have been reviewed in various papers, including one by Francesco Bascone, Franco Pezzella, and Patrizia Vitale. The twisted version of the Jacobi sigma model includes a Wess-Zumino term to accommodate twisted Jacobi backgrounds as target spaces. The models have been studied on various manifolds, including the five-dimensional sphere S5.'

In [151]:

retriever = AutoMergingRetriever(retriever ,StorageContext.from_defaults(persist_dir='./knowledge_dump/Thermodynamic_dissipation_does') ,verbose=True)

In [203]:
index,storage=create_or_load_ko_dump('darl_photon.pdf')
query_engine=get_automerging_query_engine(index,storage)